# Data Preparation

In [29]:
import pandas as pd
import numpy as np

loan_dev = pd.read_csv('data/loan_dev.csv')
client = pd.read_csv('data/client.csv')
disposition = pd.read_csv('data/disp.csv')


In [30]:
# Parse client birth number into age and gender

from datetime import datetime

def cal_age(birthday):
    today = datetime.today()
    years = today.year - birthday.year
    if today.month < birthday.month or (today.month == birthday.month and today.day < birthday.day):
        years -= 1
    return years

def parse_birthNumber(birthNumber):
    year = int('19' + str(birthNumber)[0:2]) if int(str(birthNumber)[0:2]) > 22 else int('20' + str(birthNumber)[0:2]) # Limits birth year between 1923 - 2022 (Change this if needed)
    month = int(str(birthNumber)[2:4])
    day = int(str(birthNumber)[-2:])
    gender = ''
    if month > 12:
        month -= 50
        gender ='Female'
    else: 
        gender = 'Male'

    return cal_age(datetime(year, month, day)), gender


genders = []
ages = []

for birthNumber in client['birth_number']:
    age, gender = parse_birthNumber(birthNumber)
    genders.append(gender)
    ages.append(age)

client['age'] = ages
client['gender'] = genders

client = client.drop('birth_number', axis=1)


,client_id,district_id,age,gender
0,1,18,51,Female
1,2,1,77,Male
2,3,1,81,Female
3,4,5,65,Male
4,5,5,62,Female
...,...,...,...,...
5364,13955,1,76,Female
5365,13956,1,79,Male
5366,13968,61,54,Male
5367,13971,67,59,Female


In [31]:
data = pd.merge(client, disposition, left_on="client_id", right_on="client_id", how='inner')

np.where(pd.isnull(data))

data2 = pd.merge(loan_dev, data, left_on="account_id", right_on="account_id", how='inner')
data2



,loan_id,account_id,date,amount,duration,payments,status,client_id,district_id,age,gender,disp_id,type
0,5314,1787,930705,96396,12,8033,-1,2166,30,75,Female,2166,OWNER
1,5316,1801,930711,165960,36,4610,1,2181,46,54,Male,2181,OWNER
2,6863,9188,930728,127080,60,2118,1,11314,45,86,Male,11006,OWNER
3,5325,1843,930803,105804,36,2939,1,2235,14,82,Female,2235,OWNER
4,7240,11013,930906,274740,60,4579,1,13539,63,44,Male,13231,OWNER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,6818,9030,961212,155616,48,3242,1,11122,72,43,Male,10814,DISPONENT
399,5625,3189,961215,222180,60,3703,-1,3855,29,83,Male,3855,OWNER
400,6805,8972,961221,45024,48,938,1,11050,70,65,Female,10742,OWNER
401,7233,10963,961225,115812,36,3217,1,13480,16,69,Male,13172,OWNER


In [ ]:
loan_dev.to_csv('data_processed/data.csv')